In [42]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, GridSearchCV
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU
import numpy as np
import time
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from keras_regressor import compileModel

In [37]:
df = pd.read_csv("../data/totaldf.csv", index_col='request_date')
df.head()

,requests,tempC,precipMM,WindGustKmph
request_date,,,,
2015-09-01 00:00:00,708,16.0,0.0,17.0
2015-09-01 01:00:00,479,16.0,0.0,17.0
2015-09-01 02:00:00,492,15.0,0.0,18.0
2015-09-01 03:00:00,563,15.0,0.0,18.0
2015-09-01 04:00:00,355,16.0,0.0,15.0


In [43]:
indexCol = 'request_date'
stepsIn = 504
stepsOut = 168
testsize =stepsIn+stepsOut

In [45]:
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, 0]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

computeCols = [col for col in df.columns if col!=indexCol]
df[computeCols] = df[computeCols].astype('float32')

# Scale & Formulate as a Supervised  Learning method
scalers = []
df2 = pd.DataFrame(columns = df.columns, index=df.index)
for col in df.columns:
    scaler = StandardScaler()
    df2[col] = scaler.fit_transform(np.array(df[col]).reshape(-1, 1))
    scalers.append(scaler)

scaler = scalers[0]
data = np.array(df2)
X, Y = split_sequences(data, stepsIn, stepsOut)

# Train/set split
trainsize = len(df2)-testsize
train_X = X[:trainsize, :]
train_Y = Y[:trainsize]
test_X = X[trainsize:, :]
test_Y = Y[trainsize:]
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], df2.shape[1]))   # reshape for LSTM input
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], df2.shape[1]))
inputshape = train_X.shape

# Train Best model!

In [ ]:
best_params_cv = {'L2': 0, 'alpha': 0.1, 'batch_size': 128, 'dropout': 0.3, 
                  'epochs': 10, 'learning_rate': 0.01, 'optimizer': 'adam'}   # discovered through GridSearchCV

l2_reg = 0
alpha = 0.1
batch_size = 128
dropout = 0.3
epochs = 10
learning_rate = 0.01
optimizer = 'adam'

In [48]:
def compileModel(epochs, l2_reg, alpha, batch_size, dropout, 
    learning_rate, optimizer, init='glorot_uniform'):
    input_shape = (2233, 504, 4) 
    model = Sequential()                           # LSTM input layer MUST be 3D - (samples, timesteps, features)
    model.add(LSTM(504,
                #return_sequences=True,          # necessary for stacked LSTM layers
            input_shape=(input_shape[1], input_shape[2])))
    #model.add(LSTM(10))
    model.add(Dropout(dropout))
    model.add(Dense(256, 
            kernel_initializer = init,
            kernel_regularizer=l2(l2_reg), 
            activation=LeakyReLU(alpha=alpha)))
    model.add(Dropout(dropout))
    model.add(Dense(168))

    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

model = compileModel(epochs, l2_reg, alpha, batch_size, dropout, learning_rate, optimizer)

model.fit(train_X, train_Y, epochs=epochs, batch_size=batch_size)

Train on 2233 samples
Epoch 1/10
2233/2233 [==============================] - 866s 388ms/sample - loss: 0.5116 - mean_absolute_error: 0.5116
Epoch 2/10
2233/2233 [==============================] - 884s 396ms/sample - loss: 0.4633 - mean_absolute_error: 0.4633
Epoch 3/10
2233/2233 [==============================] - 1025s 459ms/sample - loss: 0.4492 - mean_absolute_error: 0.4492
Epoch 4/10
2233/2233 [==============================] - 1017s 455ms/sample - loss: 0.4385 - mean_absolute_error: 0.4385
Epoch 5/10
2233/2233 [==============================] - 992s 444ms/sample - loss: 0.4278 - mean_absolute_error: 0.4278
Epoch 6/10
2233/2233 [==============================] - 1008s 451ms/sample - loss: 0.4242 - mean_absolute_error: 0.4242
Epoch 7/10
2233/2233 [==============================] - 994s 445ms/sample - loss: 0.4170 - mean_absolute_error: 0.4170
Epoch 8/10
2233/2233 [==============================] - 998s 447ms/sample - loss: 0.4109 - mean_absolute_error: 0.4109
Epoch 9/10
2233/2233 [=

In [49]:
pred_Y = model.predict(test_X)
test_score = round(mean_absolute_error(test_Y, pred_Y),3)
print("Test MAE: {s}".format(s=test_score))

Test MAE: 0.6790000200271606


In [52]:
model.save('./final_keras.h5')

# GridSearchCV

In [12]:
parallel = 2
paramGrid = {
        'epochs': [5, 10],
        'L2': [0, 0.1, 0.3],
        'alpha': [0.01, 0.1, 0.3],
        'batch_size': [64, 128],
        'dropout': [0.3, 0.5, 0.1],
        'learning_rate': [0.01, 0.001],
        'optimizer': ['adam']
      }

combinations = list(itertools.product(*list(paramGrid.values())))
print(len(combinations))

2


In [13]:
modelname = 'keras_regressor.h5'
paramname = 'keras_regressor_best_params.json'

start = time.time()

paramGrid = {
        'epochs': [10],
        'L2': [0],#, 0.1, 0.3],
        'alpha': [0.1, 0.3],
        'batch_size': [128],
        'dropout': [0.3],
        'learning_rate': [0.01],
        'optimizer': ['adam']
      }

k_fold = KFold(n_splits=2)   
combinations = list(itertools.product(*list(paramGrid.values())))


grid = GridSearchCV(KerasRegressor(build_fn=compileModel, 
                                    verbose=1), 
                    param_grid=paramGrid, 
                    scoring='neg_mean_absolute_error',            # ignore minority class size, treat as equal
                    n_jobs=parallel, 
                    return_train_score=True, 
                    refit='neg_mean_absolute_error',              # refit based on score of f1_scorer
                    cv=k_fold) 

grid_result = grid.fit(train_X, train_Y)
    
bestParameters = grid_result.best_params_
print("Best parameter set found:", bestParameters)

best_model = grid_result.best_estimator_.model
best_model_history = grid_result.best_estimator_.model.history.history

pred_Y = best_model.predict(test_X)

print("Best KerasRegressor MAE score {s}".format(s=mean_absolute_error(test_Y, pred_Y)))

end = time.time()
elapsed = round(end-start,3)
print("Elapsed time: {s} seconds for {c} models".format(s=elapsed, c=len(combinations)))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2233 samples
Epoch 1/10
2233/2233 [==============================] - 248s 111ms/sample - loss: 0.5106 - mean_absolute_error: 0.5106
Epoch 2/10
2233/2233 [==============================] - 360s 161ms/sample - loss: 0.4631 - mean_absolute_error: 0.4631
Epoch 3/10
2233/2233 [==============================] - 404s 181ms/sample - loss: 0.4452 - mean_absolute_error: 0.4452
Epoch 4/10
2233/2233 [==============================] - 456s 204ms/sample - loss: 0.4326 - mean_absolute_error: 0.4326
Epoch 5/10
2233/2233 [==============================] - 585s 262ms/sample - loss: 0.4299 - mean_absolute_error: 0.4299
Epoch 6/10
2233/2233 [==============================] - 712s 319ms/sample - loss: 0.4195 - mean_absolute_error: 0.4195
Epoch 7/10
2233/2233 [==============================] - 709s 317ms/sample - loss: 0.4136 

In [14]:
best_model.save('./gscv_best_lstm.h5')